NBA teams are increasingly trotting out lineups with five players who can play and guard nearly any position. traditional positions don't accurately explain what a players skillset truly is, they incorrectly oversimplify the skill sets of NBA players. Simply plugging players into one of five positions does not accurately define a player’s specific skill set. Moreover, the misclassification of a player’s position may lead teams to waste resources on developing draft picks that do not fit their systems.

In light of these changes, we need an effective way to designate positions in the NBA not based on basic physical traits such as height and weight, but in terms of function, such as shooting and defense. A framework for modern NBA positions is important towards our understanding for how players have evolved, and effective roster construction. 

### Import the Required Python Packages and Methods

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import csv


### Finding and Evaluating Historical NBA Player Data

I will use the Kaggle dataset "NBA Players stats since 1950", with stats for all players since 1950. The file Seasons_Stats.csv contains the statics of all players since 1950, with details of each column as below:

    Year - Season
    Player - name
    Pos - Position
    Age - Age
    Tm - Team
    G - Games
    GS - Games Started
    MP - Minutes Played
    PER - Player Efficiency Rating
    TS% - True Shooting %
    3PAr - 3-Point Attempt Rate
    FTr - Free Throw Rate
    ORB% - Offensive Rebound Percentage
    DRB% - Defensive Rebound Percentage
    TRB% - Total Rebound Percentage
    AST% - Assist Percentage
    STL% - Steal Percentage
    BLK% - Block Percentage
    TOV% - Turnover Percentage
    USG% - Usage Percentage
    blanl
    OWS - Offensive Win Shares
    DWS - Defensive Win Shares
    WS - Win Shares
    WS/48 - Win Shares Per 48 Minutes
    blank2
    OBPM - Offensive Box Plus/Minus
    DBPM - Defensive Box Plus/Minus
    BPM - Box Plus/Minus
    VORP - Value Over Replacement
    FG - Field Goals
    FGA - Field Goal Attempts
    FG% - Field Goal Percentage
    3P - 3-Point Field Goals
    3PA - 3-Point Field Goal Attempts
    3P% - 3-Point Field Goal Percentage
    2P - 2-Point Field Goals
    2PA - 2-Point Field Goal Attempts
    2P% - 2-Point Field Goal Percentage
    eFG% - Effective Field Goal Percentage
    FT - Free Throws
    FTA - Free Throw Attempts
    FT% - Free Throw Percentage
    ORB - Offensive Rebounds
    DRB - Defensive Rebounds
    TRB - Total Rebounds
    AST - Assists
    STL - Steals
    BLK - Blocks
    TOV - Turnovers
    PF - Personal Fouls
    PTS - Points

In [7]:
with open('data/Seasons_Stats.csv','rb') as csvfile:
    reader1 = csv.reader(csvfile)
df1 = pd.read_csv('data/Seasons_Stats.csv')

First, we remove duplicated or empty rows and drop a couple of blank columns.

In [8]:
df1_no_duplicates = df1.drop_duplicates()
df2 = df1_no_duplicates
del df2['blanl']
del df2['blank2']
df2.dropna(how='all')
df2.head(5)

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


A second file, players.csv, contains static information for each player, as height, weight, etc.

In [9]:
players = pd.read_csv('data/Players.csv', index_col=0)
players.head(5)

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


I then selected these three data categories:
    
    Player - Player's full name (first and last)
    height - Height in cm
    weight - Weight in kg

1.The players have unique names (checked at the beginning), we can merge the two dataframes using the 'Player' column.

In [10]:
data = pd.merge(df2, players[['Player', 'height', 'weight']], left_on='Player', right_on='Player', right_index=False,
      how='left', sort=False)
#.fillna(value=0)

2.Using rate statistics (i.e. points per game) or cumulative statistics (i.e. total points) can be misleading when it comes to analysis because these statistics tend to inflate players with lengthier careers. To deal with outliers, I instituted a minimum threshold of 40 games played. Also I keep only players with more than 400 minutes for each season (with a 82 games regular season, thats around 5 minutes per game. Players with less than that will be only anecdotical, and will distort the analysis).

In [11]:
data = data[~(data['Pos']==0) & (data['MP'] > 400) & (data['G'] > 40)]
data.reset_index(inplace=True, drop=True)

Let's have a look at the Player position labels:

In [12]:
data.Pos.value_counts()

PF       3171
SF       3066
SG       3044
PG       2972
C        2960
SF-SG      22
C-PF       21
SG-PG      17
PG-SG      17
SG-SF      17
PF-C       16
PF-SF      15
SF-PF      13
F-C         9
G-F         6
C-F         3
SG-PF       3
G           3
C-SF        2
F           2
F-G         2
SF-PG       1
PG-SF       1
Name: Pos, dtype: int64

Let's keep only the 5 'single' positions as there is not enough data for the players who can have two different positions.¶

In [13]:
data = data[data.Pos.isin(['PF', 'PG', 'C', 'SG', 'SF'])]

3.Replace the * sign in some of the names.

In [14]:
data['Player'] = data['Player'].str.replace('*','')

4.For the stats that represent total values (others, as TS%, represent percentages), we will take the values per 36 minutes. The reason is to judge every player according to his characteristics, not the time he was on the floor.

In [15]:
totals = ['PER', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
         'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for col in totals:
    data[col] = 36 * data[col] / data['MP']

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15213 entries, 0 to 15382
Data columns (total 53 columns):
Unnamed: 0    15213 non-null int64
Year          15213 non-null float64
Player        15213 non-null object
Pos           15213 non-null object
Age           15211 non-null float64
Tm            15213 non-null object
G             15213 non-null float64
GS            11070 non-null float64
MP            15213 non-null float64
PER           15212 non-null float64
TS%           15213 non-null float64
3PAr          11543 non-null float64
FTr           15213 non-null float64
ORB%          12868 non-null float64
DRB%          12868 non-null float64
TRB%          13409 non-null float64
AST%          14117 non-null float64
STL%          12868 non-null float64
BLK%          12868 non-null float64
TOV%          12039 non-null float64
USG%          12039 non-null float64
OWS           15212 non-null float64
DWS           15212 non-null float64
WS            15212 non-null float64
WS/48   

from the data information above, the ideal number of data rows we want to work with is 15383, most amount of data is missing from 

    3P  (3-Point Field Goals)                  11543 non-null float64
    3PA (3-Point Field Goal Attempts)          11543 non-null float64
    3P% (3-Point Field Goal Percentage)        10478 non-null float64
    3PAr(3-Point Attempt Rate)                 11543 non-null float64
    
It's becasue 3-point line was not introduced to NBA until 1979, but for the sake of classifying modern players, I have to assume all players before 1979 have not attempted 3-pointers, so I will fill the missing values of these three columns with 0. 

In addition:

    GS  (Games Started)                        11070 non-null float64

Since number of Games started for each player does not effect their performance measuring, I will delete this column.

In [17]:
threePointsCol = ['3P','3PA','3P%','3PAr']
for feature in threePointsCol:
    data[feature] = data[feature].fillna(0)
del data['GS']
data[threePointsCol].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15213 entries, 0 to 15382
Data columns (total 4 columns):
3P      15213 non-null float64
3PA     15213 non-null float64
3P%     15213 non-null float64
3PAr    15213 non-null float64
dtypes: float64(4)
memory usage: 594.3 KB


The next groups of columns that have substantical amount of missing data is from:
    
    ORB           12868 non-null float64
    DRB           12868 non-null float64
    STL           12868 non-null float64
    BLK           12868 non-null float64
    TOV           12039 non-null float64
    
And their associate percentages(plus USG%, relevant to TOV):
    
    ORB%          12868 non-null float64
    DRB%          12868 non-null float64
    STL%          12868 non-null float64
    BLK%          12868 non-null float64
    TOV%          12039 non-null float64
    USG%          12039 non-null float64
    
Since the above data has minimul 12039 or 12868 rows, the commonality is due to the fact ORB, DRB, STL, BLK and their associate percentages are not included in this dataset before 1974, and TOV data are not present until 1978. These missing values cannot be filled with assumption, but at the same time these columns are too important to be removed from the construction of player classification, therefore all the data before 1978 have to be discarded. 

In [18]:
data = data[~(data['Year'] < 1978)]

5.Cast these columns to be of type 'int'.

In [19]:
integerCol = ['Year','Age', 'G']
for feature in integerCol:
    data[feature] = data[feature].astype(dtype ='int')

In [20]:
pd.options.display.max_columns = None
data.sample(5).transpose()

,11692,8560,14480,13466,6044
Unnamed: 0,18384,13118,23160,21450,9134
Year,2007,1997,2015,2012,1989
Player,Mickael Gelabale,Travis Knight,Jonas Jerebko,Shaun Livingston,Ricky Berry
Pos,SF,C,PF,SG,SG
Age,23,22,27,26,24
Tm,SEA,LAL,TOT,MIL,SAC
G,70,71,75,58,64
MP,1239,1156,1230,1092,1406
PER,0.284746,0.4391,0.439024,0.362637,0.33542
TS%,0.535,0.536,0.549,0.52,0.552


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12039 entries, 3218 to 15382
Data columns (total 52 columns):
Unnamed: 0    12039 non-null int64
Year          12039 non-null int64
Player        12039 non-null object
Pos           12039 non-null object
Age           12039 non-null int64
Tm            12039 non-null object
G             12039 non-null int64
MP            12039 non-null float64
PER           12039 non-null float64
TS%           12039 non-null float64
3PAr          12039 non-null float64
FTr           12039 non-null float64
ORB%          12039 non-null float64
DRB%          12039 non-null float64
TRB%          12039 non-null float64
AST%          12039 non-null float64
STL%          12039 non-null float64
BLK%          12039 non-null float64
TOV%          12039 non-null float64
USG%          12039 non-null float64
OWS           12039 non-null float64
DWS           12039 non-null float64
WS            12039 non-null float64
WS/48         12039 non-null float64
OBPM       

In [25]:
data.max()

Unnamed: 0                 24689
Year                        2017
Player        Zydrunas Ilgauskas
Pos                           SG
Age                           43
Tm                           WSB
G                             85
MP                          3638
PER                      1.96299
TS%                         0.73
3PAr                        0.88
FTr                        1.234
ORB%                        20.8
DRB%                          38
TRB%                        29.7
AST%                        57.5
STL%                         5.8
BLK%                        10.8
TOV%                        38.3
USG%                        41.7
OWS                        0.184
DWS                      0.10741
WS                      0.241502
WS/48                      0.325
OBPM                    0.245629
DBPM                    0.425616
BPM                     0.255118
VORP                    0.159315
FG                       12.6901
FGA                       25.393
FG%       

In [26]:
data.min()

Unnamed: 0          5010
Year                1978
Player        A.C. Green
Pos                    C
Age                   18
Tm                   ATL
G                     41
MP                   403
PER            0.0585683
TS%                0.338
3PAr                   0
FTr                0.023
ORB%                 0.2
DRB%                 2.9
TRB%                 2.4
AST%                   0
STL%                 0.1
BLK%                   0
TOV%                 2.2
USG%                 3.7
OWS           -0.0980036
DWS           -0.0199778
WS            -0.0692308
WS/48             -0.091
OBPM           -0.632432
DBPM           -0.392593
BPM             -0.74067
VORP           -0.086262
FG              0.727273
FGA              1.52727
FG%                0.282
3P                     0
3PA                    0
3P%                    0
2P             0.0582524
2PA             0.757282
2P%                0.077
eFG%               0.292
FT             0.0487145
FTA             0.146939


In [ ]:
data.to_csv('data/Seasons_Stats_cleansed.csv', sep='\t')